## Массив bin`ов (тестовый)

In [177]:
bins = [
    '004443', '220005', '220006', '220009', '220012',
    '220013', '220014', '220016', '220018', '220022',
    '220031', '220032', '220033', '220037', '220043',
    '220046', '220047', '220049', '220052', '220055',
    '220056', '220057', '220058', '220059', '220061',
    '220064', '220066', '220074', '220076', '220078',
    '220080', '220082', '220083', '220084', '220087'
]

## Вариант доступа через официальное API (ограничение: 1000/мес, далее платно, либо пишет пустоту(типо не определил))

In [136]:
import requests

file_name = 'type1.txt'

url = "https://bin-ip-checker.p.rapidapi.com/"
headers = {
	"content-type": "application/json",
	"X-RapidAPI-Key": "ab8b8c83d5msh780e2bc27db536cp1df682jsn76db90722239",
	"X-RapidAPI-Host": "bin-ip-checker.p.rapidapi.com"
}
result = ''

for bin in bins:
  
  payload = {"bin": bin}
  print(payload)
  
  response = requests.request("POST", url, json=payload, headers=headers, params=payload)
  
  if (response.status_code != 200):
      continue

  response = json.loads(response.text)
  
  if(response['BIN']['issuer']['name'] != '' and response['BIN']['issuer']['name'] != '------'):
    bank = response['BIN']['issuer']['name']
    country = response['BIN']['country'][ 'alpha2']
  else:
    continue

  result =  result +'insert into banks(bin, name, country) values('+bin+',\''+bank+'\',\''+country+'\'),\n'
  
print(result)


{'bin': '220065'}
{'bin': '220065'}
insert into banks(bin, name, country) values(220065,'VTB BANK OJSC','RU'),
insert into banks(bin, name, country) values(220065,'VTB BANK OJSC','RU'),



## "Хитрый" вариант

In [183]:
import requests
import json

file_name = 'type2.txt'
result = ''

for bin in bins:
  bin_url = url + bin
  page_text = requests.get(bin_url).text
  #print(page_text)
  
  #bank
  comment_index = page_text.find('//Same Country BINs')
  index_data = page_text.find('data:', comment_index)
  index_success = page_text.find('},', index_data)
  data = page_text[index_data:index_success+1]
  data = data.replace('data:', '')
  data = data.replace('issuer_name', '"issuer_name"')
  data = data.replace('country_name', '"country_name"')
  data = data.replace('\'', '"')
  data = json.loads(data)
  bank = data['issuer_name']
  #country = data['country_name']

  #country
  comment_index = page_text.find('Код страны ISO A2')
  index_data = page_text.find('class="p-2">', comment_index)
  index_success = page_text.find('</td>', index_data)
  country = page_text[index_data:index_success+1]
  country = country.replace('class="p-2">', '')
  country = country.replace('<', '')

  if(bank != ''):
    result = result + 'insert into banks(bin, name, country) values('+bin+',\''+bank+'\',\''+country+'\'),\n'

print(result)


insert into banks(bin, name, country) values(557885,'BELARUSSIAN BANK OF DEVELOPMENT AND RECONSTRUCTION BELINVESTBANK JSC','BY'),



## Печать результата в файл и выгрузка

In [182]:
from google.colab import files
f = open(file_name, 'w')
f.write(result)
f.close()
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [164]:
ls

banks_new.txt  banks.txt  sample_data/  type1.txt  type2.txt
